In [1]:
# 乗算レイヤの実装


class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

In [2]:
# 加算レイヤの実装


import numpy as np
import matplotlib.pylab as plt


class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

- 図5-17の実装

In [3]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
## リンゴ一つの値段と個数を掛け算
apple_price = mul_apple_layer.forward(apple, apple_num)
## オレンジ一つの値段と個数を掛け算
orange_price = mul_orange_layer.forward(orange, orange_num)
## オレンジとリンゴの値段の和
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
## 消費税率をかける
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# 活性化関数レイヤの実装

In [4]:
# ReLUレイヤの計算グラフ


class ReLU:
    def __init__(self):
        self.mask = None  # True/FalseからなるNumpy配列
        # xも同様にTrue/Falseからなる配列

    def forward(self, x: np.ndarray):
        """順伝播(じゅんでんぱ)"""
        self.mask = (
            x <= 0
        )  # xの値の中で、0以下の値の場所(インデックスとはちょっと違う)をTrueにし、それ以外はFalseにする。
        out = (
            x.copy()
        )  # copyメソッドを使用してから複製を行うことで、参照先のメモリアドレスが違う場所になる。
        out[self.mask] = 0  # 0以下の部分を0にする

        return out

    def backward(self, dout):
        """逆伝播(ぎゃくでんぱ)"""
        dout[self.mask] = 0  # 0以下の部分を0にする。
        dx = dout

        return dx

- シグモイド関数 $ y = \frac{1}{1 + e^{-x}} $

In [5]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out  # インスタンス変数へ保存

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

# Affine/Softmaxレイヤの実装
今までやってきたニューラルネットワークの順電波の行列積は、幾何学分野では「アフィン変換」と呼ばれる。

$ Y = X ・ W + B$


In [6]:
import numpy as np


class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        # 重みバイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        out = np.dot(x, self.W) + self.b
        self.x = x  # インスタンス変数へ保存

        return out

    def backward(self, dout):
        # この処理で何をしているのかがわからない
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### ソフトマックス関数と損失関数を結合したレイヤの構成

In [7]:
import sys, os
import numpy as np

sys.path.append(os.pardir)
from common.functions import cross_entropy_error, softmax


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 損失
        self.y = None  # ソフトマックス関数の出力
        self.t = None  # 教師データ(one-hot形式)

    def forward(self, x: np.ndarray, t: np.ndarray):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)  # 損失関数の計算

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

### 誤差逆伝播法を実装した、ニューラルネットワークの実装

In [8]:
import sys, os
import numpy as np
from common.layers import *
from common.gradient import (
    numerical_gradient,
)  # 数値微分によって損失関数の勾配を求める関数
from collections import OrderedDict  # 順序付き辞書型クラス


class TwoLayerNet:
    """2層ニューラルネットワーク"""

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みとバイアスの初期化
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)  # このコードの意味がわからない
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers["Affine1"] = Affine(self.params["W1"], self.params["b1"])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W2"], self.params["b2"])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        # 順伝播
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """最後のレイヤは損失関数のレイヤ"""
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        """あるループ間でのモデル精度を計算"""
        y = self.predict(x)
        y = np.argmax(y, axis=1)

        # 以下のif文による処理の解説 :
        # ndimは、np.ndarray型の変数tの次元数を参照している
        # これが1出なかったとき、つまり多次元配列出なかった場合のみ,
        # np.argmaxで配列の中で最も値が大きい数値のインデックスを参照するメソッドを呼び出している.
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        """数値微分による勾配計算

        Args:
            x (numpy.ndarray): _description_
            y (numpy.dnarray): _description_
        """
        loss_W = lambda W: self.loss(
            x, t
        )  # loss_Wという関数を形式的に定義,Wはダミー変数で使われることはない

        grads = {}
        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])

        return grads

    def gradient(self, x, t):
        """誤差逆伝播による勾配計算

        Args:
            x (numpy.ndarray): _description_
            t (numpy.ndarray): _description_
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1  # 初期値
        dout = self.lastLayer.backward(dout)  # 損失関数の逆伝播

        layers = list(self.layers.values())
        layers = reversed(layers)  # レイヤ順序反転

        # 逆伝播
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads["W1"], grads["b1"] = self.layers["Affine1"].dW, self.layers["Affine1"].db
        grads["W2"], grads["b2"] = self.layers["Affine2"].dW, self.layers["Affine2"].db

        return grads

        # なぜ、数値微分は計算に時間がかかるの?
        # ここがそもそもわからない.

### 勾配確認

**勾配確認**とは,誤差逆伝播法によって求められた勾配と、数値微分によって求められた勾配の差分を比較し、
誤差逆伝播法が正しく実装されているかを確認する作業である。


In [16]:
import sys, os

sys.path.append(os.pardir)
from dataset.mnist import load_mnist

# from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist()

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)  # Backpropagation:誤差逆伝播法の意

# 各重みの絶対誤差の平均値を求める
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(f"{key}:{str(diff)}")

W1:3.996743932387138e-10
b1:2.663029389046072e-09
W2:6.244039074375628e-09
b2:1.402270424805119e-07


# 誤差逆伝播法による、学習!

In [23]:
import sys, os
import numpy as np

sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000  # ここは適宜変更
train_size = x_train.shape[0]

batch_size = 100
learning_rate = 0.1

train_loss_list = []  # 損失関数の値のリスト
train_acc_list = []  # 訓練データによるモデル精度のリスト
test_acc_list = []  # テストデータによるモデル精度のリスト

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(
        train_size, batch_size
    )  # 60000からbatch_size(多分100)個のデータを無作為抽出
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 誤差逆電波法によって勾配を求める
    grad = network.gradient(x_batch, t_batch)

    # update hyper parameters
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1epochごとの処理
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09736666666666667 0.0982
0.7824833333333333 0.7889
0.8753833333333333 0.8783
0.89815 0.9004
0.9081 0.9099
0.9143166666666667 0.9157
0.9192666666666667 0.9213
0.92305 0.9256
0.9279 0.9298
0.93055 0.9326
0.9340166666666667 0.9345
0.9362833333333334 0.9378
0.93805 0.9384
0.9407666666666666 0.9417
0.9422166666666667 0.9413
0.9444333333333333 0.9438
0.946 0.9458
